# Loading the data

In [1]:
import random
import json

with open('fixed_destination.json', 'r') as f:
    d = json.load(f)

maze = d['maze']
data = d['data']

'''
model data is 2D list
inner dimension representing each data item: [0.52, 0.5, 0.1522936964597336, 'right']
'''

# shuffle data
random.shuffle(data)
data[:10]

[[0.5, 0.52, 0.02503692947096968, 'down'],
 [0.5, 0.52, 0.02503692947096968, 'down'],
 [0.52, 0.5, 0.1583474454905175, 'right'],
 [0.5, 0.52, 0.33590565746973633, 'down'],
 [0.5, 0.52, 0.6019286087064073, 'down'],
 [0.52, 0.5, 0.1583474454905175, 'right'],
 [0.5, 0.48, 0.33590565746973633, 'up'],
 [0.52, 0.5, 0.49317031630635705, 'right'],
 [0.5, 0.52, 0.15022157682581808, 'down'],
 [0.52, 0.5, 0.19554467028633868, 'right']]

## Training the model:
* each data item is value like: [0.5, 0.52, 0.02503692947096968, 'down']
* first two values represent vectorized co-ordinates
* third value represent distance between current location and destination
* last value represent label for model output

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# # Assuming your data is a list of lists
# # Each data item is of the form: [x1, x2, distance, label]
# # Example data
# data = [
#     [0.5, 0.52, 0.02503692947096968, 'down'],
#     # Add more data items here
# ]

# Extract features (first three values) and labels
X = np.array([item[:3] for item in data])
y = np.array([item[3] for item in data])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Example usage to predict a new instance
new_instance = np.array([[0.6, 0.48, 0.030012, 'up']])
prediction = model.predict(new_instance[:, :3])
print(f"Prediction: {prediction}")
